In [28]:
import csv
import time
import numpy as np 
from sklearn.ensemble import AdaBoostClassifier
from sklearn import linear_model

def normalize(x):
    '''This function nomalizes each columns of the input 2d array.'''
    x_mean = np.mean(x, axis=0)
    x_std = np.std(x, axis=0)
    x_std[x_std == 0] = 1
    x1 = (x - x_mean) / x_std
    return x1

def addFeature (X_train, X_test1, X_test2, add_feature):
    addFeatureLen = len(add_feature)
    for i1 in range (addFeatureLen):
        for i2 in range (i1,addFeatureLen, 1):
            newFeature1 = np.array(X_train[:,i1]*X_train[:,i2]).reshape(-1,1)
            #print (np.shape(newFeature1))
            X_train = np.hstack((X_train, newFeature1))
            newFeature2 = np.array(X_test1[:,i1]*X_test1[:,i2]).reshape(-1,1)
            X_test1 = np.hstack((X_test1, newFeature2))
            newFeature3 = np.array(X_test2[:,i1]*X_test2[:,i2]).reshape(-1,1)
            X_test2 = np.hstack((X_test2, newFeature3))
    return (X_train, X_test1, X_test2)

def selectFeature(X_train, y_train, x_test1, x_test2,alpha1):
    '''This function select the features of normalized data (i.e., np.std(X[:,j]) = 1 or 0).
    If qmin < (np.amax(X[:,j]) - np.amin(X[:,j]) < qmax, then j will be selected.'''
    reg = linear_model.Lasso(alpha = alpha1)
    reg.fit(X_train, y_train)
    keeplist = []
    feature_num = len(reg.coef_)
    for i in range(feature_num):
        if abs(reg.coef_[i])> 1e-4:
            keeplist.append(True)
        else:
            keeplist.append(False)
    keeplist[0] = True
    keeplist = np.array(keeplist)
    #print (reg.coef_[[0, 6, 7, 39, 41]])
    #for index, value in enumerate(keeplist):
    #    if value:
    #        print (index)
    #print (reg.coef_[52])
    #dist = np.amax(X_train, axis=0) - np.amin(X_train, axis=0)
    #cols = np.all([dist > qmin, dist < qmax], axis=0)
    x_train_new = X_train[:, keeplist]
    x_test1_new = X_test1[:, keeplist]
    x_test2_new = X_test2[:, keeplist]
    return (x_train_new, x_test1_new, x_test2_new)

start = time.time()

# load the the data from the files
with open('train_2008.csv', 'r') as file1: 
    lines1 = csv.reader(file1, delimiter=',', quotechar='|') 
    next(lines1, None)
    data1 = np.array([line for line in lines1], dtype=float)

with open('test_2008.csv', 'r') as file2:
	lines2 = csv.reader(file2, delimiter=',', quotechar='"')
	next(lines2, None)
	data2 = np.array([line for line in lines2], dtype=float)

with open('test_2012.csv', 'r') as file3:
	lines3 = csv.reader(file3, delimiter=',', quotechar='"')
	next(lines3, None)
	data3 = np.array([line for line in lines3], dtype=float)

# convert the data to float numpy array 
alpha1 = 0.065
add_feature = [6,7,39,41,48,57,64,371,374];
N_train = len(data1)
y_train = 2 * (data1[:, -1] - 1.5)  # maps 1 to -1, 2 to 1
X_train = normalize(data1[:, :-1])
X_train[:, 0] = 1
X_test1 = normalize(data2)
X_test1[:, 0] = 1
X_test2 = normalize(data3)
X_test2[:, 0] = 1
#qmin, qmax = 1, 100
#X_train, X_test1, X_test2 = addFeature(X_train, X_test1, X_test2, add_feature)
X_train, X_test1, X_test2 = selectFeature(X_train, y_train, X_test1, X_test2, alpha1) 
print (np.shape(X_train))
d = len(X_train[0])
print (d)




(64667, 5)
5


In [96]:
#with open('train_2008.csv', 'r') as file1: 
#    lines1 = csv.reader(file1, delimiter=',', quotechar='|') 
#    data1 = np.array([line for line in lines1])


In [98]:
#data1[0,[64,337,371,374]]
#data1[0,[6,7,39,41,48,57,64,371,374]]


array(['HETENURE', 'HEHOUSUT', 'PERRP', 'PEAGE', 'PEEDUCA', 'PRMARSTA',
       'PRINUSYR', 'PECYC', 'PEDISREM', 'PEDISOUT'], 
      dtype='<U9')

In [21]:
# train the model and calculate the scores by cross-validation
N = 200
clf1 = AdaBoostClassifier(n_estimators=N)
clf1.fit(X_train[:int(N_train/2)], y_train[:int(N_train/2)])
score1 = clf1.score(X_train[int(N_train/2):], y_train[int(N_train/2):])
clf2 = AdaBoostClassifier(n_estimators=N)
clf2.fit(X_train[int(N_train/2):], y_train[int(N_train/2):])
score2 = clf2.score(X_train[:int(N_train/2)], y_train[:int(N_train/2)])
print("The cross-validation scores are : ", score1, score2)

#train on the whole sample
clf = AdaBoostClassifier(n_estimators=N)
clf.fit(X_train, y_train)
score3 = clf.score(X_train[:int(N_train/2)], y_train[:int(N_train/2)])
print("The cross-validation scores are : ", score3)

# write the prediction data into the submission file
#clf = AdaBoostClassifier(n_estimators=N)
#clf.fit(X_train, y_train)
'''
y_test1 = clf2.predict(X_test1)
print([sum(y_test1==-1), sum(y_test1==1)])
with open('submission2008.csv', 'w', newline='') as file: 
	filewriter = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
	filewriter.writerow(['id', 'PES1'])
	for i, yi in enumerate(y_test1):
		filewriter.writerow([str(i), str(int(yi/2 + 1.5))])
y_test2 = clf2.predict(X_test2)
print([sum(y_test2==-1), sum(y_test2==1)])
with open('submission2012.csv', 'w', newline='') as file: 
	filewriter = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
	filewriter.writerow(['id', 'PES1'])
	for i, yi in enumerate(y_test2):
		filewriter.writerow([str(i), str(int(yi/2 + 1.5))])
'''
y_test1 = clf.predict(X_test1)
print([sum(y_test1==-1), sum(y_test1==1)])
with open('submission2008.csv', 'w', newline='') as file: 
	filewriter = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
	filewriter.writerow(['id', 'PES1'])
	for i, yi in enumerate(y_test1):
		filewriter.writerow([str(i), str(int(yi/2 + 1.5))])
y_test2 = clf.predict(X_test2)
print([sum(y_test2==-1), sum(y_test2==1)])
with open('submission2012.csv', 'w', newline='') as file: 
	filewriter = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
	filewriter.writerow(['id', 'PES1'])
	for i, yi in enumerate(y_test2):
		filewriter.writerow([str(i), str(int(yi/2 + 1.5))])
# print running time
stop = time.time()
print('The running time is ', stop - start)




The cross-validation scores are :  0.776705634935 0.782203940247
The cross-validation scores are :  0.787090588563
[13773, 2227]
[73617, 9203]
The running time is  79.94712996482849


In [ ]:
non:
    0.776520071751 0.780286394705
    
select feature:
The cross-validation scores are :  0.776705634935 0.782203940247

add and select
The cross-validation scores are :  0.777880868436 0.78294621594
The cross-validation scores are :  0.787090588563
[13731, 2269]
[73258, 9562]
The running time is  137.72293400764465

In [24]:
(0.776520071751 + 0.780286394705)/2-(0.776705634935+0.782203940247)/2
(0.776705634935+0.782203940247)/2-(0.777880868436+0.78294621594)/2

-0.0009587545970000111

The cross-validation scores are :  0.776148945383 0.781709089784
The cross-validation scores are :  0.787090588563
[13703, 2297]
[70848, 11972]
The running time is  100.95253396034241

The cross-validation scores are :  0.775654110225 0.78310085671
The cross-validation scores are :  0.786533881793
[13737, 2263]
[70926, 11894]
The running time is  102.10718703269958
0.0095:
The cross-validation scores are :  0.776489144554 0.781307023784
The cross-validation scores are :  0.786162743946
[13768, 2232]
[71027, 11793]
The running time is  89.1989848613739

The cross-validation scores are :  0.776520071751 0.780286394705
The running time is  84.21992111206055

alpha = 0.002; 152 feature selected:
The cross-validation scores are :  0.776736562133 0.781399808245
The running time is  61.33863091468811

alpha = 0.005;  97 feature selected:
The cross-validation scores are :  0.776860270922 0.782420437324
The running time is  46.867387771606445

alpha = 0.0075; 68 feature selected:
The cross-validation scores are :  0.776550998948 0.782853431479
The running time is  42.11497497558594

alpha = 0.01; 55 feature selected:
The cross-validation scores are :  0.776241726975 0.781863730554
The running time is  39.134161949157715

In [5]:
x = np.array([1,2,3])

In [8]:
index = np.array([True,False,True])
x[index]

array([1, 3])

In [11]:
x[:-1]

array([1, 2])

In [42]:
x[2]=5

In [43]:
x

array([1, 2, 5])

In [111]:
m = range(0, 9, 3)

In [112]:
len(m)

3

In [114]:
m[2]

6